In [0]:
!pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.0.0)


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%cd drive/'My Drive'/repositories/comp551p3/

[Errno 2] No such file or directory: 'drive/My Drive/repositories/comp551p3/'
/content/drive/My Drive/551P3


In [0]:
#Zhenghua (Mavis) test---DON'T RUN THIS
%cd drive/'My Drive'/551P3/

[Errno 2] No such file or directory: 'drive/My Drive/551P3/'
/content/drive/My Drive/551P3


In [0]:
%pwd


'/content/drive/My Drive/551P3'

# Introduction


# Data

In [0]:
import numpy as np
import pandas as pd
train_y = pd.read_csv('data/train_max_y.csv', index_col=0).values
train_x = np.load('data/train_max_x', allow_pickle=True).astype(np.uint8) / 255

FileNotFoundError: ignored

In [0]:
#Mavis test---DON'T RUN THIS
import numpy as np
import pandas as pd
train_y = pd.read_csv('train_max_y.csv', index_col=0).values
train_x = np.load('train_max_x', allow_pickle=True).astype(np.uint8) / 255

In [0]:
import matplotlib.pyplot as plt
plt.bar(np.arange(10), np.bincount(train_y.reshape(-1)))
plt.show()

## Data augmentation

We need to fix the imbalance in the dataset

In [0]:
from scipy import ndimage
count_max = (train_y == 9).sum()
for i in range(10):
  count = (train_y == i).sum()
  

In [0]:
import matplotlib.pyplot as plt
plt.hist(train_x.reshape(-1), bins=50, histtype='step', density=True)
plt.show()

In [0]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_x[i], cmap=plt.cm.binary)
    plt.xlabel(train_y[i])
plt.show()

In [0]:
train_x_clipped = train_x.copy()
train_x_clipped[train_x < 0.92] = 0

In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_x_clipped[i], cmap=plt.cm.binary)
    plt.xlabel(train_y[i])
plt.show()

In [0]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(train_y),train_y)

In [0]:
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((train_x.reshape(-1, 128, 128, 1), train_y))
train_dataset.element_spec

In [0]:
from tensorflow.keras import Sequential, layers

def make_model():
  model = Sequential([
      layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128,1)),
      layers.MaxPool2D((3,3)),
      layers.Conv2D(64, (3,3), activation='relu'),
      layers.Conv2D(64, (3,3), activation='relu'),
      layers.MaxPool2D((3,3)),
      layers.Dropout(0.5),
      layers.Flatten(),
      layers.Dense(1024, activation='relu'),
      layers.Dense(10, activation='softmax')
  ])

  return model

In [0]:
from tensorflow.keras.callbacks import EarlyStopping
earlystop_callback = EarlyStopping(
  monitor='val_accuracy', min_delta=0.001,
  patience=2)

In [0]:
from tensorflow.keras import optimizers
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.00001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [0]:
from tensorflow.keras import callbacks
import os
checkpoint_path = "checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint_callback = callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)

In [0]:
history = model.fit(train_x.reshape(-1, 128, 128, 1), train_y, epochs=12, callbacks=[checkpoint_callback])

In [0]:
history = model.fit(train_x_clipped.reshape(-1, 128, 128, 1), train_y, epochs=12, callbacks=[checkpoint_callback])

In [0]:
history = model.fit(train_x.reshape(-1, 128, 128, 1), train_y, epochs=12, class_weight=class_weights, callbacks=[checkpoint_callback])